## __Transfer Learning__
- Transfer learning refers to a technique in machine learning where a pre-trained model, typically trained on a large dataset, is used as a starting point for solving a different but related task.
- It involves using models that were trained on one problem as a starting point for solving a related problem.
- It is flexible, allowing the use of pre-trained models directly, as feature extraction preprocessing, and integrated into entirely new models.



In this demo, we will learn how to utilize transfer learning with the VGG16 model to adapt pre-trained features for a new classification task, highlighting efficient model adaptation without extensive new training.

## Steps to be followed:
1. Import the required libraries
2. Add classifier layers
3. Perform preprocessing and feature extraction

### Step 1: Import the required libraries

- The **from tensorflow.keras.utils import load_img** loads an image file from the file system.

- The **from tensorflow.keras.utils import img_to_array** converts an image loaded with load_img into a NumPy array.

- The **from keras.applications.vgg16 import preprocess_input** preprocesses the input image array before feeding it to the VGG16 model. VGG16 expects the input images to be preprocessed in a specific way.

- The **from keras.applications.vgg16 import VGG16** imports the VGG16 model architecture. VGG16 is a popular convolutional neural network model pre-trained on the ImageNet dataset for image classification.

In [1]:
!pip install tensorflow==2.17.0 scikeras==0.13.0 keras==3.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

# Disable oneDNN optimizations to avoid potential minor numerical differences caused by floating-point round-off errors.
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [3]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input

from keras.applications.vgg16 import VGG16


2024-11-06 10:03:34.489516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 10:03:34.504195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 10:03:34.508582: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 10:03:34.519768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 10:03:36.708777: W ten

### Step 2: Add classifier layers
- It demonstrates how to load a pre-trained VGG16 model without its classifier layers and then add new custom classifier layers on top of it.
- The new model is defined by connecting the output of the pre-trained VGG16 model to a flattening layer. It is followed by a dense layer with 1024 units and ReLU activation and a dense layer with 10 units and Softmax activation for multi-class classification.
- The model summary provides an overview of the architecture and layer configurations.

In [ ]:

from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

base_model = VGG16(include_top=False, input_shape=(300, 300, 3))
flat1 = Flatten()(base_model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)

model = Model(inputs=base_model.inputs, outputs=output)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 300, 300, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 300, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 150, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 150, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 150, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 75, 75, 128)       0   

**Observation**
- Running the example means that the new model is ready for training and summarizes the model architecture.
- The output of the last pooling layer is flattened, and the new fully connected layers are added.
- The weights of the VGG16 model and the new model will all be trained together on the new dataset.

### Step 3: Perform preprocessing and feature extraction
- The image is loaded from a file and preprocessed to meet the input requirements of the VGG16 model (resizing, converting to a numpy array, and reshaping).

- The modified model predicts and extracts features from the input image, resulting in a feature vector with a specific shape.

In [ ]:
image = load_img('dog.jpg', target_size=(300, 300))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

# Predict using the correct model
features = model.predict(image)
print("Output shape:", features.shape)

1/1 [==============================] - 1s 1s/step
Output shape: (1, 10)


**Observation**

- The VGG16 model weights are downloaded and loaded successfully, and the extracted features from the input image have a shape of (1, 10).
- The final layer in the model is a Dense layer with 10 units and a softmax activation function. This is designed to output the probabilities of each of the 10 classes for the input image. Since you have one image as input, the batch size is 1, leading to the output shape of (1, 10).